In [3]:
from datasets import load_dataset
from transformers import T5Tokenizer,T5ForConditionalGeneration
import nltk
nltk.download('punkt_tab')
from rouge_score import rouge_scorer
#下载nltk需要的资源
nltk.download('punkt')
import sentencepiece


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# 加载 CNN/Daily Mail 数据集
dataset = load_dataset("cnn_dailymail",'1.0.0')  # 尝试其他版本,三番五次！最终确定 1.0.0!!!

# 查看数据集结构
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [23]:
# 定义一个文本预处理函数
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())  # 小写化并分词
    return ' '.join(tokens)

# 应用预处理函数
dataset = dataset.map(lambda x: {'article': preprocess_text(x['article']),
                                'highlights': preprocess_text(x['highlights'])})
    #太慢了。。。换成多进程
# 使用多进程处理数据
#dataset = dataset.map(
  #  lambda x: {'article': preprocess_text(x['article']),
   #            'highlights': preprocess_text(x['highlights'])},
  #  num_proc=2  # 使用4个进程并行处理    

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
# 删除或注释掉之前的预处理代码

# 然后加载保存的数据集
from datasets import load_from_disk
dataset = load_from_disk('processed_dataset')

In [8]:

# 加载 T5 模型和分词器
model_name = "t5-small"  # 选择一个较小的模型以提高训练速度
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

C:\Users\Admin\.conda\envs\text-summarization\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
# 定义一个生成摘要的函数
def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 选择数据集中的一篇文章进行测试
sample_text = dataset['train'][0]['article']
generated_summary = summarize(sample_text)
print("Generated Summary: ", generated_summary)

Generated Summary:  daniel radcliffe says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. he says he is keeping his feet firmly on the ground. radcliffe's earnings from the first five potter films have been held in a trust fund which he has not touched.


In [10]:
# 定义一个评估函数
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def evaluate_summary(generated, reference):
    return scorer.score(reference, generated)

# 获取真实摘要
reference_summary = dataset['train'][0]['highlights']
# 评估生成的摘要
evaluation = evaluate_summary(generated_summary, reference_summary)
print("ROUGE Scores: ", evaluation)

ROUGE Scores:  {'rouge1': Score(precision=0.5294117647058824, recall=0.6923076923076923, fmeasure=0.5999999999999999), 'rouge2': Score(precision=0.42, recall=0.5526315789473685, fmeasure=0.4772727272727273), 'rougeL': Score(precision=0.5098039215686274, recall=0.6666666666666666, fmeasure=0.5777777777777778)}


In [15]:
sample_text = dataset['train'][1]['article']  # 获取训练集中的第二篇文章


In [16]:
print("Original Article: ", sample_text)

Original Article:  editor 's note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o'brien takes users inside a jail where many of the inmates are mentally ill. an inmate housed on the `` forgotten floor , '' where many mentally ill inmates are housed in miami before trial . miami , florida ( cnn ) -- the ninth floor of the miami-dade pretrial detention facility is dubbed the `` forgotten floor . '' here , inmates with the most severe mental illnesses are incarcerated until they 're ready to appear in court . most often , they face drug charges or charges of assaulting an officer -- charges that judge steven leifman says are usually `` avoidable felonies . '' he says the arrests often result from confrontations with police . mentally ill people often wo n't do what they 're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more 

In [19]:
print("Generated Summary: ", generated_summary)


Generated Summary:  daniel radcliffe says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. he says he is keeping his feet firmly on the ground. radcliffe's earnings from the first five potter films have been held in a trust fund which he has not touched.


In [1]:
import os
print(os.getcwd())

C:\Users\Admin
